In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from gzip import open

import pandas as pd

import kraft

In [ ]:
file_path = "/home/kwat/CarrotCake/garden/data/medulloblastoma/northcott_et_al/GSE37382_family.soft.gz"

In [ ]:
with open(file_path) as io:

    platform_data_headers = []

    platform_data_rows = []

    samples = {}

    for line in io:

        line = line.decode()

        if line == "!platform_table_begin\n":

            line = io.readline().decode()

            while line != "!platform_table_end\n":

                platform_data_rows.append(line.rstrip().split(sep="\t"))

                line = io.readline().decode()

        if line.startswith("^SAMPLE"):

            sample = line.rstrip().split(sep=" = ")[1]

            samples[sample] = {"table": []}

            line = io.readline().decode()

            while line.startswith("^SAMPLE"):

                if line == "!sample_table_begin\n":

                    line = io.readline().decode()

                    while line != "!sample_table_end\n":

                        samples[sample]["table"].append(line.rstrip().split(sep="\t"))

                        line = io.readline().decode()

                else:

                    try:

                        key, value = line.rstrip().split(sep=" = ")

                    except:

                        continue

                    samples[sample][key] = value

                line = io.readline().decode()

platform_table = pd.DataFrame(data=platform_data_rows)

platform_table.columns = platform_table.iloc[0]

assert not platform_table.columns.has_duplicates

platform_table.drop(0, inplace=True)

platform_table.set_index("ID", inplace=True)

assert not platform_table.index.has_duplicates

platform_table